In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

data = pd.read_csv('../data_cmaps/train_FD001.txt', sep=' ', header=None)

# print(data.shape)
# print(data)
# @todo do not normalize index column of course :)
#data = normalize(data, axis=1)
# print(data)

labels = pd.read_csv('../data_cmaps/RUL_FD001.txt', header=None)
train_y = labels[0]
print(train_y.shape)
print(train_y[:10])


(100,)
0    112
1     98
2     69
3     82
4     91
5     93
6     91
7     95
8    111
9     96
Name: 0, dtype: int64


In [8]:
max_time = 100

series = [k.as_matrix()[:max_time] for i,k in pd.DataFrame(data).groupby(0)]
series = np.stack(series)
# remove device column and extra columns
series = np.delete(series, [0, 26, 27], axis=2)
print(series[:3].head())

# print(train_x.shape)
# train_x = np.reshape(train_x, (train_x.shape[0] * train_x.shape[1], -1))
# print(train_x.shape)
# #train_x = normalize(train_x, axis=1)
# print(train_x.shape)
# train_x = np.reshape(train_x, (100,100,25))
# print(train_x.shape)
# train_x

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [3]:
# split test / training set
test_x = train_x[:30]
test_y = train_y[:30]
train_x = train_x[30:]
train_y = train_y[30:]

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Masking
from keras.optimizers import RMSprop
from keras import backend as k
from sklearn.preprocessing import normalize

# Start building our model
model = Sequential()

# Mask parts of the lookback period that are all zeros (i.e., unobserved) so they don't skew the model
# model.add(Masking(mask_value=0., input_shape=(max_time, 25)))

# LSTM is just a common type of RNN. You could also try anything else (e.g., GRU).
model.add(LSTM(20, input_shape=(max_time, 25), activation='relu'))

# We need 2 neurons to output Alpha and Beta parameters for our Weibull distribution
model.add(Dense(1))

# Apply the custom activation function mentioned above
# model.add(Activation(activate))

# Use the discrete log-likelihood for Weibull survival data as our loss function
model.compile(loss='mean_squared_error', optimizer=RMSprop(lr=.001))

# Fit!
model.fit(train_x, train_y, epochs=100, batch_size=2000, verbose=2, validation_data=(test_x, test_y))

# Make some predictions and put them alongside the real TTE and event indicator values
test_predict = model.predict(test_x)
test_predict = np.resize(test_predict, (30, 1))
test_result = np.concatenate((np.reshape(test_y, (30,1)), test_predict), axis=1)

# TTE, Event Indicator, Alpha, Beta
print(test_result)

Using TensorFlow backend.


Train on 70 samples, validate on 30 samples
Epoch 1/100
0s - loss: 201806768.0000 - val_loss: 124486896.0000
Epoch 2/100
0s - loss: 124107032.0000 - val_loss: 96158696.0000
Epoch 3/100
0s - loss: 95822944.0000 - val_loss: 80059680.0000
Epoch 4/100
0s - loss: 79752216.0000 - val_loss: 68896240.0000
Epoch 5/100
0s - loss: 68610344.0000 - val_loss: 60218644.0000
Epoch 6/100
0s - loss: 59945880.0000 - val_loss: 53337980.0000
Epoch 7/100
0s - loss: 53086624.0000 - val_loss: 47387664.0000
Epoch 8/100
0s - loss: 47148528.0000 - val_loss: 42985112.0000
Epoch 9/100
0s - loss: 42757840.0000 - val_loss: 39128408.0000
Epoch 10/100
0s - loss: 38910312.0000 - val_loss: 35738632.0000
Epoch 11/100
0s - loss: 35531036.0000 - val_loss: 32945388.0000
Epoch 12/100
0s - loss: 32740016.0000 - val_loss: 30106708.0000
Epoch 13/100
0s - loss: 29916030.0000 - val_loss: 27829148.0000
Epoch 14/100
0s - loss: 27645620.0000 - val_loss: 25859424.0000
Epoch 15/100
0s - loss: 25682710.0000 - val_loss: 23991724.0000
Ep

/Users/wouter/anaconda2/envs/aind-dl/lib/python3.5/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [164]:
print(test_y.shape, test_predict.shape)
print(np.concatenate((np.reshape(test_y, (30,1)), test_predict), axis=1))
print(test_y, test_predict)

(30,) (30, 1)
[[  1.12000000e+02   9.20601487e-02]
 [  9.80000000e+01   9.20601487e-02]
 [  6.90000000e+01   9.20601487e-02]
 [  8.20000000e+01   9.20601487e-02]
 [  9.10000000e+01   9.20601487e-02]
 [  9.30000000e+01   9.20601487e-02]
 [  9.10000000e+01   9.20601487e-02]
 [  9.50000000e+01   9.20601487e-02]
 [  1.11000000e+02   9.20601487e-02]
 [  9.60000000e+01   9.20601487e-02]
 [  9.70000000e+01   9.20601487e-02]
 [  1.24000000e+02   9.20601487e-02]
 [  9.50000000e+01   9.20601487e-02]
 [  1.07000000e+02   9.20601487e-02]
 [  8.30000000e+01   9.20601487e-02]
 [  8.40000000e+01   9.20601487e-02]
 [  5.00000000e+01   9.20601487e-02]
 [  2.80000000e+01   9.20601487e-02]
 [  8.70000000e+01   9.20601487e-02]
 [  1.60000000e+01   9.20601487e-02]
 [  5.70000000e+01   9.20601487e-02]
 [  1.11000000e+02   9.20601487e-02]
 [  1.13000000e+02   9.20601487e-02]
 [  2.00000000e+01   9.20601487e-02]
 [  1.45000000e+02   9.20601487e-02]
 [  1.19000000e+02   9.20601487e-02]
 [  6.60000000e+01   9.2

/Users/wouter/anaconda2/envs/aind-dl/lib/python3.5/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Masking
from keras.optimizers import RMSprop
from keras import backend as k
from sklearn.preprocessing import normalize

"""
    Discrete log-likelihood for Weibull hazard function on censored survival data
    y_true is a (samples, 2) tensor containing time-to-event (y), and an event indicator (u)
    ab_pred is a (samples, 2) tensor containing predicted Weibull alpha (a) and beta (b) parameters
    For math, see https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)
"""
def weibull_loglik_discrete(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    hazard0 = k.pow((y_ + 1e-35) / a_, b_)
    hazard1 = k.pow((y_ + 1) / a_, b_)

    return -1 * k.mean(u_ * k.log(k.exp(hazard1 - hazard0) - 1.0) - hazard1)

"""
    Not used for this model, but included in case somebody needs it
    For math, see https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)
"""
def weibull_loglik_continuous(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    ya = (y_ + 1e-35) / a_
    return -1 * k.mean(u_ * (k.log(b_) + b_ * k.log(ya)) - k.pow(ya, b_))


"""
    Custom Keras activation function, outputs alpha neuron using exponentiation and beta using softplus
"""
def activate(ab):
    a = k.exp(ab[:, 0])
    b = k.softplus(ab[:, 1])

    a = k.reshape(a, (k.shape(a)[0], 1))
    b = k.reshape(b, (k.shape(b)[0], 1))

    return k.concatenate((a, b), axis=1)


"""
    Load and parse engine data files into:
       - an (engine/day, observed history, sensor readings) x tensor, where observed history is 100 days, zero-padded
         for days that don't have a full 100 days of observed history (e.g., first observed day for an engine)
       - an (engine/day, 2) tensor containing time-to-event and 1 (since all engines failed)
    There are probably MUCH better ways of doing this, but I don't use Numpy that much, and the data parsing isn't the
    point of this demo anyway.
"""
def load_file(name):
    with open(name, 'r') as file:
        return np.loadtxt(file, delimiter=',')

np.set_printoptions(suppress=True, threshold=10000)

train = load_file('../data_cmaps/train_FD001.txt')
test_x = load_file('test_x.csv')
test_y = load_file('test_y.csv')

# Combine the X values to normalize them, then split them back out
all_x = np.concatenate((train[:, 2:26], test_x[:, 2:26]))
all_x = normalize(all_x, axis=0)

train[:, 2:26] = all_x[0:train.shape[0], :]
test_x[:, 2:26] = all_x[train.shape[0]:, :]

# Make engine numbers and days zero-indexed, for everybody's sanity
train[:, 0:2] -= 1
test_x[:, 0:2] -= 1

# Configurable observation look-back period for each engine/day
max_time = 100

def build_data(engine, time, x, max_time, is_test):
    # y[0] will be days remaining, y[1] will be event indicator, always 1 for this data
    out_y = np.empty((0, 2), dtype=np.float32)

    # A full history of sensor readings to date for each x
    out_x = np.empty((0, max_time, 24), dtype=np.float32)

    for i in range(100):
        print("Engine: " + str(i))
        # When did the engine fail? (Last day + 1 for train data, irrelevant for test.)
        max_engine_time = int(np.max(time[engine == i])) + 1

        if is_test:
            start = max_engine_time - 1
        else:
            start = 0

        this_x = np.empty((0, max_time, 24), dtype=np.float32)

        for j in range(start, max_engine_time):
            engine_x = x[engine == i]

            out_y = np.append(out_y, np.array((max_engine_time - j, 1), ndmin=2), axis=0)

            xtemp = np.zeros((1, max_time, 24))
            xtemp[:, max_time-min(j, 99)-1:max_time, :] = engine_x[max(0, j-max_time+1):j+1, :]
            this_x = np.concatenate((this_x, xtemp))

        out_x = np.concatenate((out_x, this_x))

    return out_x, out_y

train_x, train_y = build_data(train[:, 0], train[:, 1], train[:, 2:26], max_time, False)
test_x = build_data(test_x[:, 0], test_x[:, 1], test_x[:, 2:26], max_time, True)[0]

train_u = np.zeros((100, 1), dtype=np.float32)
train_u += 1
test_y = np.append(np.reshape(test_y, (100, 1)), train_u, axis=1)

"""
    Here's the rest of the meat of the demo... actually fitting and training the model.
    We'll also make some test predictions so we can evaluate model performance.
"""

# Start building our model
model = Sequential()

# Mask parts of the lookback period that are all zeros (i.e., unobserved) so they don't skew the model
model.add(Masking(mask_value=0., input_shape=(max_time, 24)))

# LSTM is just a common type of RNN. You could also try anything else (e.g., GRU).
model.add(LSTM(20, input_dim=24))

# We need 2 neurons to output Alpha and Beta parameters for our Weibull distribution
model.add(Dense(2))

# Apply the custom activation function mentioned above
model.add(Activation(activate))

# Use the discrete log-likelihood for Weibull survival data as our loss function
model.compile(loss=weibull_loglik_discrete, optimizer=RMSprop(lr=.001))

# Fit!
model.fit(train_x, train_y, nb_epoch=250, batch_size=2000, verbose=2, validation_data=(test_x, test_y))

# Make some predictions and put them alongside the real TTE and event indicator values
test_predict = model.predict(test_x)
test_predict = np.resize(test_predict, (100, 2))
test_result = np.concatenate((test_y, test_predict), axis=1)

# TTE, Event Indicator, Alpha, Beta
print(test_result)